In [1]:
#Run this immediately to input the libraries we need for the coding.
from bs4 import BeautifulSoup
import requests
import xml.etree.ElementTree as ET
import os
EA ="abcdefghijklmnopqrstuvwxyz".upper()
Presidents = []
for filename in os.listdir(r"C:\Users\Billy\Downloads\S3RP 2018\Corpora\inaugural2"):
    if ".txt" in filename:
        if filename[5:] not in Presidents:
            Presidents.append(filename[5:])
            if filename[5:] == "Nixon.txt":
                Presidents.append("Ford.txt")

In [2]:
#In the event that the Battlebots code fails, this makes the guess based on frequencies with letter fragments known.
#Run this immediately, it is only used within the Battlebots code.
def frequency(text,letters,length,ahead = True,spaces=False,bg = False):
    global freq
    freq = {}
    if bg == True:
        global bigrams
        bigrams = {}
    if type(letters) == str:
        letters = [letters]
    if (ahead == True) or (ahead=="both"):
        for x in range(len(text)-1):
            if text[x:x+len(letters[0])] in letters:
                if (" " not in text[(x+len(letters[0])):(x+1+len(letters[0]))]) or (spaces == True):
                    for letter in text[(x+len(letters[0])):(x+1+len(letters[0]))]:
                        if (letter not in freq) and ((letter != " ") or (spaces == True)):
                            freq[letter] = 1
                        elif (letter in freq) and ((letter != " ") or (spaces == True)):
                            freq[letter] += 1
                        if bg == True:
                            if (text[x]+letter) not in bigrams:
                                bigrams[text[x]+letter] = 1
                            elif (text[x]+letter) in bigrams:
                                bigrams[text[x]+letter] += 1
    if (ahead == False) or (ahead=="both"):
        for x in range(length,len(text)):
            if text[x:x+len(letters[0])] in letters:
                if (" " not in text[x-length:x]) or (spaces == True):
                    for letter in text[x-length:x]:
                        if (letter not in freq) and ((letter != " ") or (spaces == True)):
                            freq[letter] = 1
                        elif (letter in freq) and ((letter != " ") or (spaces == True)):
                            freq[letter] += 1
                        if bg == True:
                            if (letter+text[x]) not in bigrams:
                                bigrams[letter+text[x]] = 1
                            elif (letter+text[x]) in bigrams:
                                bigrams[letter+text[x]] += 1
    return freq

In [3]:
Test = " ABCDEFGHIJKALMNOPQRSTUVWXYZ"
print(frequency(Test,"A",1,True,True,True))
print(frequency(Test,"A",1,False,True,True))
print(frequency(Test,"A",1,"both"))
print(frequency(Test,"",1))

{'B': 1, 'L': 1}
{' ': 1, 'K': 1}
{'B': 1, 'L': 1, 'K': 1}
{'A': 2, 'B': 1, 'C': 1, 'D': 1, 'E': 1, 'F': 1, 'G': 1, 'H': 1, 'I': 1, 'J': 1, 'K': 1, 'L': 1, 'M': 1, 'N': 1, 'O': 1, 'P': 1, 'Q': 1, 'R': 1, 'S': 1, 'T': 1, 'U': 1, 'V': 1, 'W': 1, 'X': 1, 'Y': 1}


In [34]:
f = "AE"
print(f.replace("A","") == "E")

True


In [42]:
[1]*len([1])

[1]

In [3]:
#Takes in a list of corpora
#Names allows you to name your corpora if you want(you must put them in the same order as they are listed).
#WoF=True means it can't guess a vowel first. Change it to False if you want it to be able to.
#inputphrase allows you to input your own phrase
#If thinking = True, then it will print out the letter/word/phrase frequencies
#thoughts says how many of the most frequent phrases or words you want it to show when thinking = True
#certainty is the percentage chance (from 0-1) that our phrase is correct based on the corpora for it to guess that phrase
def BattleBots(corpora = [],names = False,WoF = True,inputphrase = False,thinking = False,thoughts = 5,certainty = [1],play = False,pause = False,BGLF = [False],Hangman = False):
    """BOT guesses phrase, generates the phrase from wheel of fortune"""
    from pandas import DataFrame,option_context,Series
    from numpy import array,nan
    from random import randint
    from IPython.display import display, HTML
    if inputphrase == False:
        p = randint(0,len(Phrases))
        Phrase = " ".join(Phrases[p])
    else:
        Phrase = inputphrase.upper()
    if play == False:
        print("The Phrase is:\t" + Phrase + "\n\n")
    if len(certainty) == 1:
        certainty *= len(corpora)
    if len(BGLF) == 1:
        BGLF *= len(corpora)
    def WOFGuessE(corpora, given, Not_in = [], FullPhrase = True, Solve = False, WORD = False, guesses = 26):
        global Best_Guess
        Best_Guess = {}
        global Best_Phrase
        Best_Phrase = {}
        words = given.split()
        for i in range(len(words)):
            for j in range(len(words[i])):
                if ((words[i][j] in Not_in) == False) and (words[i][j] != "_"):
                    Not_in.append(words[i][j])
        global W
        W = []
        if FullPhrase == False:
            for x in words:
                W.append([])
        if FullPhrase == True:
            if WORD == True:
                LENGTH = len(corpora) - len(words) + 1
            else:
                LENGTH = len(corpora)
            for h in range(LENGTH):
                if WORD == True:
                    phrase = []
                    for x in range(0,len(words)):
                        phrase += corpora[h+x]
                else:
                    phrase = corpora[h]
                if len(phrase) == len(words):
                    fit = True
                    for x in range(len(phrase)):
                        if len(phrase[x]) != len(words[x]):
                            fit = False
                            break
                    if fit == True:
                        for i in range(len(words)):
                            for j in range(len(words[i])):
                                if words[i][j] != "_":
                                    if (phrase[i][j] != words[i][j]):
                                        fit = False
                                        break
                                else:
                                    if (phrase[i][j] in Not_in) and (phrase[i][j] != words[i][j]):
                                        fit = False
                                        break
                    if fit == True:
                        if Solve == False:
                            if (" ".join(phrase) not in W) and (len(W) < 3):
                                W.append(" ".join(phrase))
                            for x in range(len(phrase)):
                                for y in range(len(phrase[x])):
                                    if words[x][y] == "_":
                                        if (phrase[x][y] in Best_Guess) == False:
                                            Best_Guess[phrase[x][y]] = 1
                                        else:
                                            Best_Guess[phrase[x][y]] += 1
                            if " ".join(phrase) not in Best_Phrase:
                                Best_Phrase[" ".join(phrase)] = 1
                            else:
                                Best_Phrase[" ".join(phrase)] += 1
                        elif Solve == True:
                            guess = " ".join(phrase)
                            if (guess in Best_Guess) == False:
                                Best_Guess[guess] = 1
                            else:
                                Best_Guess[guess] += 1
        elif FullPhrase == False:
            global BWP
            BWP= {place:{} for place in range(len(words))}
            for phrase in corpora:
                for i in range(len(words)):
                    for j in range(len(phrase)):
                        if len(words[i]) == len(phrase[j]):
                            fit = True
                            for x in range(len(words[i])):
                                if words[i][x] != "_":
                                    if words[i][x] != phrase[j][x]:
                                        fit = False
                                        break
                                else:
                                    if phrase[j][x] in Not_in:
                                        fit = False
                                        break
                            if fit == True:
                                for x in range(len(words[i])):
                                    if (phrase[j][x] in Not_in) and (phrase[j][x] != words[i][x]):
                                        fit = False
                                if fit == True:
                                    if Solve == False:
                                        if (" ".join(phrase) not in Best_Phrase) and (phrase[j] not in Known.split()) and (phrase[j] not in BWP[i]):
                                            Best_Phrase[phrase[j]] = 1
                                            BWP[i][phrase[j]] = 1
                                        elif (" ".join(phrase) in Best_Phrase) and (phrase[j] not in Known.split()) and (phrase[j] in BWP[i]):
                                            Best_Phrase[phrase[j]] += 1
                                            BWP[i][phrase[j]] += 1
                                        for x in range(len(words[i])):
                                            if (words[i][x] == "_"):
                                                if (phrase[j][x] in Best_Guess) == False:
                                                    Best_Guess[phrase[j][x]] = 1
                                                else:
                                                    Best_Guess[phrase[j][x]] += 1
                                                if (phrase[j] not in W[i]) and (len(W[i]) < 3):
                                                    W[i].append(phrase[j])
                                    else:
                                        if (phrase[j] in Best_Guess) == False:
                                            Best_Guess[phrase[j]] = 1
                                        else:
                                            Best_Guess[phrase[j]] += 1
        total = 0
        for x in Best_Guess:
            total += Best_Guess[x]
        Frequencies = sorted(Best_Guess.values(), reverse = True)
        Frequencies = array(Frequencies)/total
        Letters = sorted(Best_Guess, key = Best_Guess.__getitem__, reverse = True)
        return Letters
    HangMan = {7:
    """
    ---------
    |       |
    |  RIP  |
    |       |
    |       |
    """,
    6:
    """
      ------
      |    O
      |   /|\\
      |   / \\
      | 
    -----
    """,
    5:
    """
      ------
      |    O
      |   /|\\
      |   /
      |
    -----  
    """,
    4:
    """
      ------
      |    O
      |   /|\\
      |   
      |
    -----  
    """ ,
    3:
    """
      ------
      |    O
      |   /|
      |   
      |
    -----  
    """ ,
    2:
    """
      ------
      |    O
      |    |
      |   
      |
    -----  
    """,
    1: 
    """
      ------
      |    O
      |   
      |   
      |
    -----  
    """ ,
    0: 
    """
      ------
      |    
      |   
      |   
      |
    -----  
    """         }  
    global tries
    if play == True:
        Known = ""
        for letter in Phrase:
            if letter != " ":
                Known += "_"
            else:
                Known += " "
        tries = 0
        right = 0
        wrong = 0
        index = 0
        turn = 1
        Not_in = []
        p = True
        Not_Phrase = []
        DF = {}
        while "_" in Known:
            print("Current puzzle: "+Known.replace(""," ")+"\n")
            tries += 1
            if len(Not_in) < 1:
                Frame = DataFrame({"Letters Guessed": ""},index = [1]).T
            else:
                Frame = DataFrame(sorted(Not_in), columns = ["Letters Guessed"], index = list(range(1,len(Not_in)+1))).T
            with option_context('display.max_rows', None, 'display.max_columns', None):
                display(Frame)
            best = input("What is your guess? ").upper()
            while ((len(best) == 1) and (best not in list("ABCDEFGHIJKLMNOPQRSTUVWXYZ"))) or (best in Not_in) or (best in Not_Phrase) or ((WoF == True) and (best in list("AEIOU")) and (Known.count("_") + Known.count(" ") == len(Known))):
                if (len(best) == 1) and (best not in list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")):
                    best = input("You can only put in letters from the english alphabet. Try again. ").upper()
                elif best in Not_in:
                    best = input("You cannot guess a letter you have guessed before. Try again. ").upper()
                elif best in Not_Phrase:
                    best = input("You cannot guess a phrase you have guessed before. Try again. ").upper()
                elif ((WoF == True) and (best in list("AEIOU")) and (Known.count("_") + Known.count(" ") == len(Known))):
                    best = input("You cannot guess a vowel before a consonant. Try again. ").upper()
            if len(best) > 1:
                print("\nI would like to solve!\n\n")
                if best == Phrase:
                    DF[tries] = {"Right":"*","Wrong":""}
                    right += 1
                    print("Correct!\n"+Phrase+"\n\n")
                    break
                else:
                    DF[tries] = {"Right":"","Wrong":"*"}
                    wrong += 1
                    print("Incorrect Guess:  "+best+"\n\n")
                    Not_Phrase.append(best)
            else:
                if best not in Phrase:
                    DF[tries] = {"Right":"","Wrong":best}
                    wrong += 1
                    print("Phrase: "+Known.replace(""," ")+"\tWRONG: "+best+"\n\n")
                    if Hangman == True:
                        print(HangMan[wrong])
                        if wrong == 7:
                            print("You lose")
                    Not_in.append(best)
                else:
                    DF[tries] = {"Right":best,"Wrong":""}
                    for i in range(len(Phrase)):
                        if Phrase[i] == best:
                            Known = Known[:i] + best + Known[i+1:]
                    right += 1
                    print("Phrase: "+Known.replace(""," ")+"\tCORRECT: "+best+"\n\n")
                    Not_in.append(best)
                if WoF == True:
                    vl = []
                    for v in list("AEIOU"):
                        if v in Phrase:
                            if v not in Not_in:
                                vl.append(v)
                    if (len(vl) == 0) and p == True:
                        p = False
                        print("All vowels have been guessed\n\n")
                        for v in list("AEIOU"):
                            if v not in Not_in:
                                Not_in.append(v)
        print("Guessing Breakdown:")
        with option_context('display.max_rows', None, 'display.max_columns', None):
            display(DataFrame(DF))
        print("\nIt took you %d guess(es) to solve the puzzle, with %d correct and %d incorrect.\n\n\n" % (tries,right,wrong))
    for x in range(0,len(corpora)):
        if names == False:
            print("\t\tCorpus %d:\n\n" % (x+1))
        else:
            print("\t\t" + names[x] + ":\n\n")
        Known = ""
        for letter in Phrase:
            if letter != " ":
                Known += "_"
            else:
                Known += " "
        tries = 0
        right = 0
        wrong = 0
        index = 0
        turn = 1
        Not_in = []
        if (corpora[x]) == (Phrases):
            WORD = False
        else:
            WORD = True
        FP = True
        Fail = False
        p = True
        Not_Phrase = []
        DF = {}
        if BGLF[x] == "BG":
            FP = False
            Fail = "BG"
        elif BGLF[x] == "LF":
            FP = False
            Fail = "LF"
        if Fail == False:
            while "_" in Known:
                print("Current puzzle: "+Known.replace(""," ")+"\n")
                best = WOFGuessE(corpora[x], Known, Not_in = Not_in, WORD = WORD, FullPhrase = FP)[:]
                if (FP == True) and (Fail == False):
                    Solved = False
                    if len(Best_Phrase) >= 1:
                        Totals = sorted(Best_Phrase.values(), reverse = True)
                        Organized = sorted(Best_Phrase, key = Best_Phrase.__getitem__, reverse = True)
                        for a in range(len(Totals)):
                            Tot = array(Totals[a:])/sum(Totals[a:])
                            if thinking == True:
                                if len(Tot[:thoughts]) < 1:
                                    add = len(Tot[:thoughts]) + 1
                                    Frame = DataFrame({"Frequencies": 0,"Phrases": ""},index=list(range(1,add+1))).T
                                    with option_context('display.max_rows', None, 'display.max_columns', None):
                                        display(Frame)
                                else:
                                    add = len(Tot[:thoughts])
                                    Frame = DataFrame({"Frequencies": Tot[:thoughts],"Phrases": Organized[a:][:thoughts]},index=list(range(1,add+1))).T
                                    with option_context('display.max_rows', None, 'display.max_columns', None):
                                        display(Frame)
                            if (pause == True):
                                input("Press enter to continue...")
                            if (Tot[0] >= certainty[x]) and (Organized[a] not in Not_Phrase):
                                tries += 1
                                print("\nI would like to solve!\n\n")
                                if Organized[a] == Phrase:
                                    DF[tries] = {"Right":"*","Wrong":""}
                                    right += 1
                                    print(Phrase+"\n\n")
                                    Solved = True
                                    break
                                else:
                                    DF[tries] = {"Right":"","Wrong":"*"}
                                    wrong += 1
                                    print("Incorrect Guess:  "+Organized[a]+"\n\n")
                                    if Hangman == True:
                                        print(HangMan[wrong])
                                        if wrong == 7:
                                            print("You lose")
                                            break
                                    Not_Phrase.append(Organized[a])
                                    if a == len(Totals)-1:
                                        FP = False
                                        print("\n*Switching to word frequencies*\n\n")
                                        best = WOFGuessE(corpora[x], Known, Not_in = Not_in, WORD = WORD, FullPhrase = FP)[:]
                            else:
                                break
                        if Solved == True:
                            break
                    else:
                        FP = False
                        print("\n*Switching to word frequencies*\n\n")
                        best = WOFGuessE(corpora[x], Known, Not_in = Not_in, WORD = WORD, FullPhrase = FP)[:]
                if (FP == False) and (Fail == False):
                    for firstcase in BWP:
                        for copy in BWP:
                            if (Known.split()[firstcase] == Known.split()[copy]) and (firstcase != copy) and (BWP[copy] == {}):
                                BWP[copy] = BWP[firstcase]
                    FullList = [sorted(BWP[ww], key = BWP[ww].__getitem__, reverse = True) for ww in BWP]
                    NL = [[] for L in FullList]
                    for q in range(len(FullList)):
                        if (FullList[q] == []) and ("_" not in Known.split()[q]):
                            FullList[q].append(Phrase.split()[q])
                            BWP[q][Phrase.split()[q]] = 1
                        if q == 0:
                            for firstword in FullList[q]:
                                if BWP[q][firstword]/sum(sorted(BWP[q].values(),reverse = True)) >= certainty[x]:
                                    NL[q].append(firstword)
                        else:
                            for otherword in FullList[q]:
                                if BWP[q][otherword]/sum(sorted(BWP[q].values(),reverse = True)) >= certainty[x]:
                                    for J in NL[q-1]:
                                        NL[q].append(J + " " + otherword)
                    if thinking == True:
                        add = 0
                        for q in range(len(FullList)):
                            if len(FullList[q][:thoughts]) < 1:
                                add = len(FullList[q][:thoughts]) + 1
                            else:
                                if len(FullList[q][:thoughts]) > add:
                                    add = len(FullList[q][:thoughts])
                        Frame = DataFrame({"Best guess for word "+str(q+1): Series(FullList[q][:thoughts],index=list(range(1,len(FullList[q][:thoughts])+1))) for q in range(len(FullList))},index=list(range(1,add+1))).T
                        Frame = Frame.replace(nan,"")
                        with option_context('display.max_rows', None, 'display.max_columns', None):
                            display(Frame)
                            if len(NL[len(NL)-1][:thoughts]) < 1:
                                display(DataFrame({"Phrases that work": ""},index = [1]).T)
                            else:
                                add = len(NL[len(NL)-1][:thoughts])
                                display(DataFrame({"Phrases that work": NL[len(NL)-1][:thoughts]}, index = list(range(1,add+1))).T)
                    BetterDict = {}
                    for place in range(len(NL[len(NL)-1])):
                        for word in NL[len(NL)-1][place].split():
                            if word not in Best_Phrase:
                                BetterDict[word] = 1
                            else:
                                BetterDict[word] = Best_Phrase[word]
                    Dictionary = {NL[len(NL)-1][s]:sum(BetterDict[t] for t in NL[len(NL)-1][s].split()) for s in range(len(NL[len(NL)-1]))}
                    if len(Dictionary) >= 1:
                        Totals = sorted(Dictionary.values(), reverse = True)
                        Organized = sorted(Dictionary, key = Dictionary.__getitem__, reverse = True)
                        for a in range(len(Totals)):
                            Tot = array(Totals[a:])/sum(Totals[a:])
                            if thinking == True:
                                if len(Tot[:thoughts]) < 1:
                                    add = len(Tot[:thoughts]) + 1
                                    Frame = DataFrame({"Frequencies": 0,"Phrases": ""},index=list(range(1,add+1))).T
                                    with option_context('display.max_rows', None, 'display.max_columns', None):
                                        display(Frame)
                                else:
                                    add = len(Tot[:thoughts])
                                    Frame = DataFrame({"Frequencies": Tot[:thoughts],"Phrases": Organized[a:][:thoughts]},index=list(range(1,add+1))).T
                                    with option_context('display.max_rows', None, 'display.max_columns', None):
                                        display(Frame)
                            if (pause == True):
                                input("Press enter to continue...")
                            if (Tot[0] >= certainty[x]) and (Organized[a] not in Not_Phrase):
                                tries += 1
                                print("\nI would like to solve!\n")
                                if Organized[a] == Phrase:
                                    DF[tries] = {"Right":"*","Wrong":""}
                                    right += 1
                                    print(Phrase+"\n\n")
                                    Solved = True
                                    break
                                else:
                                    DF[tries] = {"Right":"","Wrong":"*"}
                                    wrong += 1
                                    print("Incorrect Guess:  "+Organized[a]+"\n")
                                    if Hangman == True:
                                        print(HangMan[wrong])
                                        if wrong == 7:
                                            print("You lose")
                                            break
                                    if Known.count("_") == 1:
                                        Not_in.append(Organized[a][Known.find("_")])
                                    Not_Phrase.append(Organized[a])
                                    best = WOFGuessE(corpora[x], Known, Not_in = Not_in, WORD = WORD, FullPhrase = FP)[:]
                            else:
                                break
                        if Solved == True:
                            break
                if Fail == False:
                    try:
                        Best = best[index]
                    except:
                        if FP == True:
                            print("*Switching to word frequencies*\n")
                        FP = False
                        best = WOFGuessE(corpora[x], Known, Not_in = Not_in, WORD = WORD, FullPhrase = FP)[:]
                        try:
                            Best = best[index]
                        except:
                            print("\n\n*Switching to Bigrams*\n\n")
                            Fail = "BG"
                            break
                    if (turn == 1) and (WoF == True):
                        while Best in list("AEIOU"):
                            index += 1
                            Best = best[index]
                    if (thinking == True) and (FP == False):
                        Organized = sorted(Best_Phrase, key = Best_Phrase.__getitem__, reverse = True)
                        Frame = DataFrame(Organized[:thoughts],columns = ["Most Frequent Words"], index = list(range(1,len(Organized[:thoughts])+1))).T
                        with option_context('display.max_rows', None, 'display.max_columns', None):
                            display(Frame)
                    for i in range(len(W)):
                        if len(W[i]) == 1:
                            for letter in W[i][0]:
                                if letter not in Known:
                                    if (turn == 1) and (WoF == True):
                                        if letter not in list("AEIOU"):
                                            Best = letter
                                    else:
                                        Best = letter
                                    break
                    if (pause == True):
                        input("Press enter to continue...")
                    if Best not in Phrase:
                        DF[tries+1] = {"Right":"","Wrong":Best}
                        wrong += 1
                        print("Phrase: "+Known.replace(""," ")+"\tWRONG: "+Best+"\n\n")
                        if Hangman == True:
                            print(HangMan[wrong])
                            if wrong == 7:
                                print("You lose")
                                break
                        Not_in.append(Best)
                        if turn == 1:
                            while Best in list("AEIOU"):
                                index += 1
                                Best = best[index]
                            turn = 0
                    else:
                        DF[tries+1] = {"Right":Best,"Wrong":""}
                        for i in range(len(Phrase)):
                            if Phrase[i] == Best:
                                Known = Known[:i] + Best + Known[i+1:]
                        right += 1
                        print("Phrase: "+Known.replace(""," ")+"\tCORRECT: "+Best+"\n\n")
                        Not_in.append(Best)
                    if Fail != False:
                        break
                    index = 0
                    turn += 1
                    tries += 1
                    if WoF == True:
                        vl = []
                        for v in list("AEIOU"):
                            if v in Phrase:
                                if v not in Not_in:
                                    vl.append(v)
                        if (len(vl) == 0) and p == True:
                            p = False
                            print("All vowels have been guessed\n\n")
                            for v in list("AEIOU"):
                                if v not in Not_in:
                                    Not_in.append(v)
        if Fail == "BG":
            text = " " + " ".join([corpora[x][i][0] for i in range(len(corpora[x]))]) + " "
            if thinking == True:
                BG = True
            else:
                BG = False
            while "_" in Known:
                K2 = " " + Known + " "
                if WoF == True:
                    vl = []
                    for v in list("AEIOU"):
                        if v in Phrase:
                            if v not in Not_in:
                                vl.append(v)
                    if (len(vl) == 0) and p == True:
                        p = False
                        print("\nAll vowels have been guessed\n\n")
                        for v in list("AEIOU"):
                            if v not in Not_in:
                                Not_in.append(v)
                Before = []
                After = []
                for i in range(1,len(K2)-1):
                    if K2[i] == "_":
                        if (K2[i-1] != "_") and (K2[i-1] not in After):
                            After.append(K2[i-1])
                        if (K2[i+1] != "_") and (K2[i+1] not in Before):
                            Before.append(K2[i+1])
                TBG = {}
                BD = {}
                BDl = frequency(text,Before,1,False,bg=BG)
                BD = { k: BD.get(k, 0) + BDl.get(k, 0) for k in set(BD) | set(BDl) }
                if thinking == True:
                    TBG = { k: TBG.get(k, 0) + bigrams.get(k, 0) for k in set(TBG) | set(bigrams) }
                AD = {}
                ADl = frequency(text,After,1,True,bg=BG)
                AD = { k: AD.get(k, 0) + ADl.get(k, 0) for k in set(AD) | set(ADl) }
                if thinking == True:
                    TBG = { k: TBG.get(k, 0) + bigrams.get(k, 0) for k in set(TBG) | set(bigrams) }
                TD = { k: AD.get(k, 0) + BD.get(k, 0) for k in set(AD) | set(BD) }
                if (" " in Before) and (" " in After):
                    for let in TD:
                        TD[let] -= text.count(" "+let+" ")
                Totals = sorted(TD.values(), reverse = True)
                Organized = sorted(TD, key = TD.__getitem__, reverse = True)
                for let in Not_in:
                    while let in Organized:
                        del Totals[Organized.index(let)]
                        del Organized[Organized.index(let)]
                if thinking == True:
                    BGList = sorted(TBG, key = TBG.__getitem__, reverse = True)
                    BGR = []
                    for bigram in BGList:
                        for let in bigram:
                            if ((let in Not_in) and (let not in Known)) or ((let == " ") and (bigram.replace(let,"") in Known)) or ((let in Known) and (bigram.replace(let,"") in Known)):
                                BGR.append(bigram)
                                break
                    for bigram in BGR:
                        BGList.remove(bigram)
                    with option_context('display.max_rows', None, 'display.max_columns', None):
                        Frame = DataFrame([BGs.replace(" ","_") for BGs in BGList[:thoughts]],columns = ["Best Bigrams"], index = list(range(1,len(BGList[:thoughts])+1))).T
                        display(Frame)
                        Frame = DataFrame(Organized,columns = ["Best Guesses"], index = list(range(1,len(Organized)+1))).T
                        display(Frame)
                try:
                    index = 0
                    Best = Organized[index]
                except:
                    print("\n*Switching to letter frequencies*\n\n")
                    Fail = "LF"
                    break
                tries += 1
                if (pause == True):
                    input("Press enter to continue...")
                while (WoF == True) and (Known.count(" ")+Known.count("_") == len(Known)) and (Best in list("AEIOU")):
                    index += 1
                    Best = Organized[index]
                if (Best in Phrase) and (Best not in Known):
                    DF[tries] = {"Right":Best,"Wrong":""}
                    for i in range(len(Phrase)):
                        if Phrase[i] == Best:
                            Known = Known[:i] + Best + Known[i+1:]
                    right += 1
                    print("Phrase: "+Known.replace(""," ")+"\tRIGHT: "+Best+"\n")
                else:
                    DF[tries] = {"Right":"","Wrong":Best}
                    wrong += 1
                    print("Phrase: "+Known.replace(""," ")+"\tWRONG: "+Best+"\n")
                    if Hangman == True:
                        print(HangMan[wrong])
                        if wrong == 7:
                            print("You lose")
                            break
                Not_in.append(Best)
                if "_" not in Known:
                    Fail = False
                    break
        if Fail == "LF":
            text = " ".join([corpora[x][i][0] for i in range(len(corpora[x]))])
            F = frequency(text,"",1)
            Letters = " ".join(sorted(F, key = F.__getitem__, reverse = True))
            for let in Not_in:
                Letters = Letters.replace(let+" ","")
            Letters = Letters.split()
            if thinking == True:
                with option_context('display.max_rows', None, 'display.max_columns', None):
                    Frame = DataFrame(Letters,columns = ["Most Frequent Letters"], index = list(range(1,len(Letters)+1))).T
                    display(Frame)
            for letter in Letters:
                if (letter not in Not_in) and ((WoF == False) or ((letter not in list("AEIOU")))):
                    tries += 1
                    if (pause == True):
                        input("Press enter to continue...")
                    if (letter in Phrase) and (letter not in Known):
                        DF[tries] = {"Right":letter,"Wrong":""}
                        for i in range(len(Phrase)):
                            if Phrase[i] == letter:
                                Known = Known[:i] + letter + Known[i+1:]
                        right += 1
                        print("Phrase: "+Known.replace(""," ")+"\tRIGHT: "+letter+"\n")
                    else:
                        DF[tries] = {"Right":"","Wrong":letter}
                        wrong += 1
                        print("Phrase: "+Known.replace(""," ")+"\tWRONG: "+letter+"\n")
                        if Hangman == True:
                            print(HangMan[wrong])
                            if wrong == 7:
                                print("You lose")
                                break
                    Not_in.append(letter)
                    break
            for letter in Letters:
                if ((Hangman == True) and (wrong == 7)) or ("_" not in Known):
                    break
                if WoF == True:
                    vl = []
                    for v in list("AEIOU"):
                        if v in Phrase:
                            if v not in Not_in:
                                vl.append(v)
                    if (len(vl) == 0) and p == True:
                        p = False
                        print("\nAll vowels have been guessed\n\n")
                        for v in list("AEIOU"):
                            if v not in Not_in:
                                Not_in.append(v)
                if (letter not in Not_in):
                    tries += 1
                    if (pause == True):
                        input("Press enter to continue...")
                    if (letter in Phrase) and (letter not in Known):
                        DF[tries] = {"Right":letter,"Wrong":""}
                        for i in range(len(Phrase)):
                            if Phrase[i] == letter:
                                Known = Known[:i] + letter + Known[i+1:]
                        right += 1
                        print("Phrase: "+Known.replace(""," ")+"\tRIGHT: "+letter+"\n")
                    else:
                        DF[tries] = {"Right":"","Wrong":letter}
                        wrong += 1
                        print("Phrase: "+Known.replace(""," ")+"\tWRONG: "+letter+"\n")
                        if Hangman == True:
                            print(HangMan[wrong])
                            if wrong == 7:
                                print("You lose")
                                break
                    Not_in.append(letter)
        print("Guessing Breakdown:")
        with option_context('display.max_rows', None, 'display.max_columns', None):
            display(DataFrame(DF))
        if (Hangman == False) or ((Hangman == True) and (wrong < 7)):
            if names == False:
                print("\nIt took \"Corpus %d\" %d guess(es) to solve the puzzle, with %d correct and %d incorrect.\n\n\n" % (x+1,tries,right,wrong))
            else:
                print("\nIt took " + "\"" + names[x] + "\"" + " %d guess(es) to solve the puzzle, with %d correct and %d incorrect.\n\n\n" % (tries,right,wrong))
        else:
            if names == False:
                print("\n\"Corpus %d\" failed to solve the puzzle, with %d correct guesses and %d incorrect out of %d total guesses.\n\n\n" % (x+1,right,wrong,tries))
            else:
                print("\n" + "\"" + names[x] + "\"" + " failed to solve the puzzle, with %d correct and %d incorrect out of %d total guesses.\n\n\n" % (right,wrong,tries))
        Fail = False
        Word = True
    return None

In [55]:
BattleBots([PresidentsW["Obama"]],names=["Obama"],play=False,Hangman=True,BGLF=["BG"],thinking=True,pause=True,WoF = False)

The Phrase is:	ROCKEFELLER CENTER


		Obama:




,1,2,3,4,5
Best Bigrams,E_,_T,S_,_A,D_


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
Best Guesses,T,E,S,O,D,A,N,R,W,F,Y,H,L,I,C,B,G,M,P,U,K,J,V,Q


Press enter to continue...
Phrase:  _ _ _ _ _ _ _ _ _ _ _   _ _ _ T _ _ 	RIGHT: T



,1,2,3,4,5
Best Bigrams,E_,TH,S_,_A,D_


...

,1,2,3,4,5
Best Bigrams,OU,_C,_B,_P,LE


,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Best Guesses,M,C,U,L,B,P,G,V,Y,K,J,Q,X,Z


Press enter to continue...
Phrase:  R O _ _ E F E _ _ E R   _ E N T E R 	WRONG: M


    ---------
    |       |
    |  RIP  |
    |       |
    |       |
    
You lose
Guessing Breakdown:


,1,2,3,4,5,6,7,8,9,10,11,12,13
Right,T,E,,R,,,N,,,O,F,,
Wrong,,,S,,A,I,,W,H,,,D,M



"Obama" failed to solve the puzzle, with 6 correct and 7 incorrect out of 13 total guesses.





In [29]:
BattleBots([Phrases,PresidentsW["Obama"],Shakespeare],thinking=True,certainty = [.05],inputphrase="GREEN TEA LATTE")

The Phrase is:	GREEN TEA LATTE


		Corpus 1:


Current puzzle:  _ _ _ _ _   _ _ _   _ _ _ _ _ 



,1,2,3,4,5
Frequencies,0.125,0.09375,0.09375,0.09375,0.0625
Phrases,TRACK AND FIELD,NEVER SAY NEVER,DOING THE TANGO,ROUGH AND TOUGH,MASTS AND SAILS



I would like to solve!


Incorrect Guess:  TRACK AND FIELD




,1,2,3,4,5
Frequencies,0.107143,0.107143,0.107143,0.0714286,0.0714286
Phrases,NEVER SAY NEVER,DOING THE TANGO,ROUGH AND TOUGH,MASTS AND SAILS,THERE SHE BLOWS



I would like to solve!


Incorrect Guess:  NEVER SAY NEVER




...

,1,2,3,4,5
Best Bigrams,Y_,O_,AT,H_,IT


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Best Guesses,A,O,Y,I,U,H,F,M,W,K,P,C,V,X,B,Z,J


Phrase:  G R E E N   T E A   L A T T E 	RIGHT: A

Guessing Breakdown:


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
Right,T,,,,,,,,,,,,,,,E,N,L,,G,R,,A
Wrong,,*,S,*,*,*,*,*,*,*,*,*,*,*,*,,,,Q,,,D,



It took "Corpus 3" 23 guess(es) to solve the puzzle, with 7 correct and 16 incorrect.





In [ ]:
#Most common words

In [4]:
#Cleans files, run this before proceeding onward.
def cleanfileW(corpora,alphabet):
    global text
    text = corpora
    import time
    alphabet += " "
    start = time.clock()
    nonletters = ["\\x","\\","x00"]
    for character in nonletters:
        text = text.replace(character," ")
    for character in text:
        if ((character in alphabet) == False) and ((character in nonletters) == False):
            nonletters.append(character)
    text = text.replace("\'D","ED")
    for character in nonletters:
        if character == "'":
            text = text.replace(character,"")
        else:
            text = text.replace(character," ")
    end = time.clock()
    return end-start

In [5]:
#reads in and cleans out WoF Phrases, stores it as "WoF"
with open(r"C:\Users\Billy\Downloads\Corpora 7 25 2016\Corpora 7 25 2016\all_solutions.txt") as f:
    text = repr(f.read())
text = text.replace("\\n","")
text = text.replace("Â\\xa0","")
No = []
Characters = []
for x in text:
    if (x in Characters) == False:
        Characters.append(x)
Keep = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ\\t ")
for x in Characters:
    if (x in Keep) == False:
        No.append(x)
for x in No:
    text = text.replace(x,"")
text = text.split("\\t")
Phrases = []
for x in range(len(text)):
    if text[x] != "":
        Phrases.append(text[x].split())

In [6]:
#TotalW is a text of all presidents addresses
#PresidentsW is a dictionary, call the president you want if you want only their addresses.
TotalW = ""
PresidentsW = {}
Dir = r"C:\Users\Billy\Downloads\S3RP 2018\Corpora\inaugural\inaugural"
Dir2 = r"C:\Users\Billy\Downloads\S3RP 2018\Corpora\state_union\state_union"
for President in Presidents:
    T = ""
    for filename in os.listdir(Dir):
        if President in filename:
            with open(Dir + "\\" + filename) as f:
                I = f.read().upper()
            T += " " + I
    for filename in os.listdir(Dir2):
        if President[:-4] in filename:
            with open(Dir2 + "\\" + filename) as f:
                I = f.read().upper()
            T += " " + I
    cleanfileW(T,EA)
    TotalW += " " + text
    Ph = text.split()
    for x in range(len(Ph)):
        Ph[x] = [Ph[x]]
    PresidentsW[President[:-4]] = Ph
cleanfileW(TotalW,EA)
Ph = text.split()
for x in range(len(Ph)):
    Ph[x] = [Ph[x]]
TotalW = Ph

In [7]:
#stores Bible as "Bible"
url = "http://www.gutenberg.org/cache/epub/10/pg10.txt"
r = requests.get(url)
soup= BeautifulSoup(r.content,"html5lib")
soup.find_all("pre")
BibleW = soup.get_text().upper()
cleanfileW(BibleW,EA)
Bible = text.split()
for x in range(len(Bible)):
    Bible[x] = [Bible[x]]

In [8]:
#Stores Book of Mormon as "Mormon"
url = "http://www.gutenberg.org/cache/epub/17/pg17.txt"
r = requests.get(url)
soup= BeautifulSoup(r.content,"html5lib")
soup.find_all("pre")
MormonW = soup.get_text().upper()
cleanfileW(MormonW,EA)
Mormon = text.split()
for x in range(len(Mormon)):
    Mormon[x] = [Mormon[x]]

In [9]:
#Stores The Complete Works of Shakespeare.
url = "https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt"
r = requests.get(url)
soup= BeautifulSoup(r.content,"html5lib")
soup.find_all("pre")
ShakespeareW = soup.get_text().upper()
cleanfileW(ShakespeareW,EA)
Shakespeare = text.split()
for x in range(len(Shakespeare)):
    Shakespeare[x] = [Shakespeare[x]]

In [10]:
#English corpora as "English"
with open(r"C:\Users\Billy\Downloads\Corpora 7 25 2016\Corpora 7 25 2016\eng2Words.txt", encoding = "utf-8") as f:
    English = repr(f.read())
English = English.upper()
English = English[1:].split()
for x in range(len(English)):
    English[x] = [English[x]]

In [11]:
Categories = {}
for filename in os.listdir(r"C:\Users\Billy\Downloads\S3RP 2018\Corpora\Bonus Puzzle\Bonus Puzzle"):
    if ".txt" in filename:
        with open(r"C:\Users\Billy\Downloads\S3RP 2018\Corpora\Bonus Puzzle\Bonus Puzzle" +"\\"+ filename) as f:
            I = f.read()
        B = I.split("\t")
        BO = "".join(B)
        A = BO.split("\n")
        for x in range(1,len(A)-1):
            if (A[x][1].upper() == A[x][1]) and (A[x-1][1].upper() != A[x-1][1]):
                if A[x-1] not in Categories:
                    Categories[A[x-1]] = [A[x]]
                elif A[x-1] in Categories:
                    Categories[A[x-1]].append(A[x])
for cat in Categories:
    for x in range(len(Categories[cat])):
        NI = []
        for y in Categories[cat][x]:
            if y not in list("ABCDEFGHIJKLMNOPQRSTUVWXYZ "):
                NI.append(y)
            for n in NI:
                Categories[cat][x] = Categories[cat][x].replace(n,"")
        Categories[cat][x] = Categories[cat][x].split()

In [12]:
Authors= []
AuthorsW= {}
Dir4="C:\\Users\\Billy\\Downloads\\S3RP 2018\\Corpora\\authors\\"
for filename in os.listdir(Dir4):
    if "Author" in filename:
        if filename[8:] not in Authors :
            Authors.append(filename[8:])
TotalAW=""
for filename in os.listdir(Dir4):
    Q = ""
    if "Author" in filename:
        folder = (Dir4+ "\\" + filename)
        for filename2 in os.listdir(folder):
            with open (folder+ "\\" + filename2,encoding="utf-8") as f:
                G = f.read().upper()
            Q += " " + G
            TotalAW += Q
    cleanfileW(Q,EA)
    Author = text.split()
    for x in range(len(Author)):
        Author[x] = [Author[x]]
    AuthorsW[filename[8:]] = Author
cleanfileW(TotalAW,EA)
TotalAW = text.split()
for x in range(len(TotalAW)):
    TotalAW[x] = [TotalAW[x]]

In [13]:
#Tweets Converted from Twitter then stored within corpus.
Dir5=r"C:\Users\Billy\Downloads\S3RP 2018\Corpora\Twitter\twitter"
Twitters= []
TwitterW = {}
for filename in os.listdir(Dir5):
    if "Twitter" in filename:
        if filename[9:] not in Twitters:
            Twitters.append(filename[8:])
TotalTW=""
for filename in os.listdir(Dir5):
    R = ""
    if "Twitter" in filename:
        folder2 = (Dir5+ "\\" + filename)
        for filename3 in os.listdir(folder2):
            with open (folder2+ "\\" + filename3,encoding="utf-8") as f:
                G = f.read()
            R += " " + G
            cleanfileW(R,EA)
            TotalTW += " " + text
            Twitter = text.split()
            for x in range(len(Twitter)):
                Twitter[x] = [Twitter[x]]
            TwitterW[filename[9:]] = Twitter
cleanfileW(TotalTW,EA)
Twitter = text.split()
for x in range(len(Twitter)):
    Twitter[x] = [Twitter[x]]
TotalTW = Twitter

In [14]:
#show phrase if no human players
#if thinking == False, don't print "Switching to ____"
#only keep money if you solve the puzzle
#Wheel top dollar goes to 3500 in rounds 2-3 then 5000 in rounds 4+

In [15]:
#Money
#Wins (1st,2nd, and 3rd)
#Round Wins
#Bonus Puzzle wins
#Bankrupts
#Percent of correct guesses
#Overall Letter Guesses

In [ ]:
global MoneyStats, BonusStats, WinStats, SolveStats, BankruptStats, CorrectStats, LGStats

In [ ]:
#Say what letters have been guessed every time
#Add pauses